<a href="https://colab.research.google.com/github/sushmitharamk/Machine-Learning-projects/blob/main/TITANIC_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'titanic:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F3136%2F26502%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240222%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240222T002125Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D0294b5cdc3d312eacf6ad7e3740c1a42583f9224b24a9ae779a5a7e748b60d21840972c75f81724767dd31889b97292947d3dcc4e85e2a3ec1e628311ec9464cde7be96cbf6c33d1cbbe09b607f5112f3bad1cfcd64b6093a25195210b421106bae5d253cd31afb72d9d543b3394998f31a4c9eedd00d6c0e2b5ef6ad082efd83550c5d6546595f9c2e801bb9b76c28a68eaae6d68f43d306c991404b90e75e3b0740e8244f8cd9bdd0221f763b438cba508e7124f77d4df6e969609862df0d95872433f748002bf92f292a8a68105c03026ccc3a4509abe0212258e0894ea27c31f8f479aa48a3a6799a0fc003128011424287d171d5914787cabfca02b6690'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
#import libraries
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [ ]:
#load dataset
train = pd.read_csv('/kaggle/input/titanic/train.csv')
test = pd.read_csv('/kaggle/input/titanic/test.csv')

In [ ]:
#info of train data
train.info()

In [ ]:
train.describe()

In [ ]:
train.head(5)

In [ ]:
test.head(5)

In [ ]:
print('The number of duplicated data items in the train data is: ',train.duplicated().sum())

In [ ]:
#male vs female
print(train['Sex'].unique())
sns.countplot(train, x="Sex", hue='Survived')

In [ ]:
#pclass
print(train['Pclass'].unique())

sns.countplot(train,x='Pclass')

In [ ]:
d = sns.FacetGrid(train,col='Pclass', hue='Survived')
d.map(sns.countplot,'Sex')
d.add_legend()

In [ ]:
d = sns.FacetGrid(train, col = 'Pclass', hue = 'Survived')
d.map(sns.scatterplot,'Fare','Age')
d.add_legend()

In [ ]:
#check for null values:
train.isnull().sum()

In [ ]:
test.isnull().sum()

In [ ]:
#replace mean()
train['Age'] = train['Age'].fillna(train['Age'].mean())
test['Age'] = test['Age'].fillna(test['Age'].mean())
test['Fare']=test['Fare'].fillna(test['Fare'].mean())

In [ ]:
#replace Nan with mode() for categorical variable
train['Cabin'] = train['Cabin'].fillna(train['Cabin'].mode()[0])
train['Embarked'] = train['Embarked'].fillna(train['Embarked'].mode()[0])

In [ ]:
test['Cabin'] = test['Cabin'].fillna(test['Cabin'].mode()[0])
test['Embarked'] = test['Embarked'].fillna(test['Embarked'].mode()[0])

In [ ]:
train.isnull().sum()

In [ ]:
#handling categorical data
label_encoder = LabelEncoder()
train['Sex'] =label_encoder.fit_transform(train['Sex'])
train['Ticket']= label_encoder.fit_transform(train['Ticket'])
train['Cabin'] = label_encoder.fit_transform(train['Cabin'])
train['Embarked'] = label_encoder.fit_transform(train['Embarked'])

In [ ]:
test['Sex'] =label_encoder.fit_transform(test['Sex'])
test['Ticket']= label_encoder.fit_transform(test['Ticket'])
test['Cabin'] = label_encoder.fit_transform(test['Cabin'])
test['Embarked'] = label_encoder.fit_transform(test['Embarked'])

In [ ]:
train.isnull().sum()

In [ ]:
features = ['Pclass','Sex','Age','SibSp','Parch','Ticket','Fare','Cabin','Embarked']
X_train = train[features]
y_train = train['Survived']
X_test = test[features]

In [ ]:
X_train['Sex']

In [ ]:
model = RandomForestClassifier(n_estimators = 100, max_depth=5, random_state=1)
model.fit(X_train,y_train)
predictions = model.predict(X_test)

In [ ]:
passenger_id = test['PassengerId']

In [ ]:
#saving predictions to csv
prediction = pd.DataFrame(predictions, columns=['Survived'])
prediction =pd.merge(passenger_id,prediction,left_index=True, right_index =True)

In [ ]:
prediction.to_csv('output.csv', index= False)